# Example for qutrit GST
This notebook demonstrates how to construct the operation sequences and perform the analysis for qutrit GST when the model consists of symmetric $\pi/2$-rotations on each single qubit separately, `X`, `Y` and a 2-qubit Molmer-Sorenson gate which rotates around the `XX` axis by $\pi/2$.

In [1]:
import pygsti
from pygsti.models import qutrit
from pygsti.algorithms.fiducialselection import find_fiducials
from pygsti.algorithms.germselection import find_germs

from numpy import pi, array
import pickle

import numpy as np

First, we construct the target model.  This functionality is built into pyGSTi, so we just need to specify the single-qubit and M-S angles.
Note there are alternative approaches for building a qutrit model in pygsti using processor specification objects, but for this particular class of qutrit models in this example notebook there exist helper functions for creating the relevant models.

In [2]:
target_model = qutrit.create_qutrit_model(error_scale=0, x_angle=pi/2, y_angle=pi/2, ms_global=pi/2, ms_local=0, basis="qt")
#change the forward simulator for the purposes of experiment design code
target_model.sim = 'matrix'
print(target_model)

rho0 = TPState with dimension 9
 0.58-0.41   0   0   0   0 0.71   0   0


Mdefault = TPPOVM with effect vectors:
0bright: FullPOVMEffect with dimension 9
 0.58-0.41   0   0   0   0 0.71   0   0

1bright: FullPOVMEffect with dimension 9
 0.58 0.82   0   0   0   0   0   0   0

2bright: ComplementPOVMEffect with dimension 9
 0.58-0.41   0   0   0   0-0.71   0   0



Gi:QT = 
FullTPOp with shape (9, 9)
 1.00   0   0   0   0   0   0   0   0
   0 1.00   0   0   0   0   0   0   0
   0   0 1.00   0   0   0   0   0   0
   0   0   0 1.00   0   0   0   0   0
   0   0   0   0 1.00   0   0   0   0
   0   0   0   0   0 1.00   0   0   0
   0   0   0   0   0   0 1.00   0   0
   0   0   0   0   0   0   0 1.00   0
   0   0   0   0   0   0   0   0 1.00


Gx:QT = 
FullTPOp with shape (9, 9)
 1.00   0   0   0   0   0   0   0   0
   0-0.50 0.87   0   0   0   0   0   0
   0 0.87 0.50   0   0   0   0   0   0
   0   0   0-1.00   0   0   0   0   0
   0   0   0   0 1.00   0   0   0   0
   0   0   0   0   0   0-1

In [3]:
target_model.num_params

314

Now construct the operation sequences needed by GST. Then we construct an empty dataset containing all of the necessary experimental sequences which can serve as a template for the actual experimental results.

In [4]:
%%time
fiducialPrep, fiducialMeasure = find_fiducials(target_model, candidate_fid_counts={4: 'all upto'}, algorithm= 'greedy')
germs = find_germs(target_model, randomize=False, candidate_germ_counts={4: 'all upto'}, mode= 'compactEVD', assume_real=True, float_type=np.double)
maxLengths = [1,2,4]

Initial Length Available Fiducial List: 121
Length Available Fiducial List Dropped Identities and Duplicates: 50
Using greedy algorithm.
Complete initial fiducial set succeeds.
Now searching for best fiducial set.
Starting fiducial list optimization. Lower score is better.
Acceptable candidate solution found.
Score: major=-9 minor=17.99999999999997, N: 9
Exiting greedy search.
Preparation fiducials:
['{}@(QT)', 'Gm:QTGm:QT@(QT)', 'Gx:QT@(QT)', 'Gy:QT@(QT)', 'Gy:QTGy:QTGx:QT@(QT)', 'Gy:QTGy:QTGy:QT@(QT)', 'Gm:QT@(QT)', 'Gm:QTGx:QT@(QT)', 'Gm:QTGy:QT@(QT)']
Score: 17.99999999999997
Complete initial fiducial set succeeds.
Now searching for best fiducial set.
Starting fiducial list optimization. Lower score is better.
Acceptable candidate solution found.
Score: major=-9 minor=19.449999999999978, N: 9
Exiting greedy search.
Measurement fiducials:
['{}@(QT)', 'Gx:QT@(QT)', 'Gy:QT@(QT)', 'Gm:QT@(QT)', 'Gy:QTGm:QT@(QT)', 'Gm:QTGx:QT@(QT)']
Score: 19.449999999999978
Initial Length Available Ger

In [5]:
print("%d prep fiducials" % len(fiducialPrep))
print("%d meas fiducials" % len(fiducialMeasure))
print("%d germs" % len(germs))

9 prep fiducials
6 meas fiducials
9 germs


In [6]:
#generate data template
expList = pygsti.circuits.create_lsgst_circuits(target_model.operations.keys(), fiducialPrep, fiducialMeasure, germs,  maxLengths)
pygsti.io.write_empty_dataset("example_files/dataTemplate_qutrit_maxL=4.txt", expList, "## Columns = 0bright count, 1bright count, 2bright count")

At this point **STOP** and create/fill a dataset file using the template written in the above cell.  Then proceed with the lines below to run GST on the data and create (hopefully useful) reports telling you about your gates.

In [15]:
mdl_datagen = target_model.depolarize(op_noise=0.05, spam_noise = .01)
DS = pygsti.data.simulate_data(mdl_datagen, expList, 1000, sample_error='multinomial', seed=2018)

In [13]:
#DS = pygsti.io.load_dataset('PATH_TO_YOUR_DATASET',cache=True) # (cache=True speeds up future loads)

In [16]:
%%time
#Run qutrit GST... which could take a while onspam_noise=ingle CPU.  Please adjust memLimit to machine specs 
# (now 3GB; usually set to slightly less than the total machine memory)
#Setting max_iterations lower than default for the sake of the example running faster. 
target_model.sim = "matrix"
result = pygsti.run_stdpractice_gst(DS, target_model, fiducialPrep, fiducialMeasure, germs, maxLengths,
                                    verbosity=3, comm=None, mem_limit=3*(1024)**3, modes="CPTPLND",
                                    advanced_options= {'max_iterations':50})

--- Circuit Creation ---
   774 circuits created
   Dataset has 774 entries: 774 utilized, 0 requested circuits were missing
-- Std Practice:  Iter 1 of 1  (CPTPLND) --: 
    Precomputing CircuitOutcomeProbabilityArray layouts for each iteration.
    Layout for iteration 0
    Layout creation w/mem limit = 3.00GB
    MatrixLayout: 1 processors divided into 1 x 1 (= 1) grid along circuit and parameter directions.
       1 atoms, parameter block size limits (None,)
    *** Distributing 1 atoms to 1 atom-processing groups (1 cores) ***
        More atom-processors than hosts: each host gets ~1 atom-processors
        Atom-processors already occupy a single node, dividing atom-processor into 1 param-processors.
    *** Divided 1-host atom-processor (~1 procs) into 1 param-processing groups ***
       Esimated memory required = 0.1GB
    Layout for iteration 1
    Layout creation w/mem limit = 3.00GB
    MatrixLayout: 1 processors divided into 1 x 1 (= 1) grid along circuit and parameter di

    Sum of Chi^2 = 521.758 (766 data params - 432 (approx) model params = expected mean of 334; p-value = 2.03176e-10)
    Completed in 46.8s
    Iteration 2 took 46.8s
    
  --- Iterative GST: Iter 3 of 3  774 circuits ---: 
    --- chi2 GST ---


    Sum of Chi^2 = 1322.7 (1548 data params - 432 (approx) model params = expected mean of 1116; p-value = 1.70083e-05)
    Completed in 78.1s
    Iteration 3 took 78.2s
    
    Last iteration:
    --- dlogl GST ---


    2*Delta(log(L)) = 1324.81 (1548 data params - 432 (approx) model params = expected mean of 1116; p-value = 1.42741e-05)
    Completed in 17.2s
    Final optimization took 17.2s
    
CPU times: total: 1min 25s
Wall time: 2min 53s


In [10]:
#Create a report
ws = pygsti.report.construct_standard_report(
    result, "Example Qutrit Report", verbosity=3
).write_html('example_files/sampleQutritReport', auto_open=False, verbosity=3)

Running idle tomography
Computing switchable properties


C:\Users\ciostro\Documents\pyGSTi_API_updates\pygsti\report\factory.py:1284: UserWarning: Idle tomography failed:

  _warnings.warn("Idle tomography failed:\n" + str(e))


In [11]:
print(target_model)

rho0 = TPState with dimension 9
 0.58-0.41   0   0   0   0 0.71   0   0


Mdefault = TPPOVM with effect vectors:
0bright: FullPOVMEffect with dimension 9
 0.58-0.41   0   0   0   0 0.71   0   0

1bright: FullPOVMEffect with dimension 9
 0.58 0.82   0   0   0   0   0   0   0

2bright: ComplementPOVMEffect with dimension 9
 0.58-0.41   0   0   0   0-0.71   0   0



Gi:QT = 
FullTPOp with shape (9, 9)
 1.00   0   0   0   0   0   0   0   0
   0 1.00   0   0   0   0   0   0   0
   0   0 1.00   0   0   0   0   0   0
   0   0   0 1.00   0   0   0   0   0
   0   0   0   0 1.00   0   0   0   0
   0   0   0   0   0 1.00   0   0   0
   0   0   0   0   0   0 1.00   0   0
   0   0   0   0   0   0   0 1.00   0
   0   0   0   0   0   0   0   0 1.00


Gx:QT = 
FullTPOp with shape (9, 9)
 1.00   0   0   0   0   0   0   0   0
   0-0.50 0.87   0   0   0   0   0   0
   0 0.87 0.50   0   0   0   0   0   0
   0   0   0-1.00   0   0   0   0   0
   0   0   0   0 1.00   0   0   0   0
   0   0   0   0   0   0-1